# Riddler Classic - [December 6, 2019](https://fivethirtyeight.com/features/how-fast-can-you-skip-to-your-favorite-song/)

_You have a playlist with exactly 100 tracks (i.e., songs), numbered 1 to 100. To go to another track, there are two buttons you can press: (1) “Next,” which will take you to the next track in the list or back to song 1 if you are currently on track 100, and (2) “Random,” which will take you to a track chosen uniformly from among the 100 tracks. Pressing “Random” can restart the track you’re already listening to — this will happen 1 percent of the time you press the “Random” button._

_For example, if you started on track 73, and you pressed the buttons in the sequence “Random, Next, Random, Random, Next, Next, Random, Next,” you might get the following sequence of track numbers: 73, 30, 31, 67, 12, 13, 14, 89, 90. You always know the number of the track you’re currently listening to._

_Your goal is to get to your favorite song (on track 42, of course) with as few button presses as possible. What should your general strategy be? Assuming you start on a random track, what is the average number of button presses you would need to make to reach your favorite song?_

## Analytic Solution

### The stradegy

Let's not a few thing about the solution that should be true given the problem statement.  

- 1) The optimal stradegy should involve first using the 'Random' bottom some number of times followed by using 'Next' botton some number of times.  It doesn't make sense to use the 'Next' botton if you intend to use the 'Random' botton again.
- 2) When using the 'Random' botton, your assigned position won't be based off where you've been previously so your decision about when to switch to using the 'Next' botton should only depend on your current position and the expectation of how close you will get if you continue to use the 'Random' botton.

With this in mind, the optimal stradegy I propose is as follows:
- 1) Start by hitting the 'Random' botton
- 2) At each random position, calculate the probability that you can get closer by continuing to hit the 'Random' botton.  This is a little complicated to let's break this down further.
    - a) When calculating how likely it is we can get closer to the desired track using the 'Random' botton, we need to account for the fact that we could be using the 'Next' botton to guarantee we get closer to the desired track by one step.  To do this we first calculate the probabilty that the next time we press 'Random', we are assigned a track number that is less than d-1 steps away from the desired track (where d is the number of steps our current track is away from the desired track).  
    - b) Moving forward, we calculate the probabilty that if we hit 'Random' twice, the minimum distance of the two random tracks is less than d-2 steps away from the desired tracks.  
    - c) We can continue this until we calculate the probability that after selecting 'Random' d times, the minimum distance in that selection is d-d=0 steps away from the desired track (in other words one of the 'Random' selections actually landed on the desired track)
    - d) Summing all these terms together, we get the total probability that selecting 'Random' will get us closer to the desired track than beginning to hit 'Next'
- 3) Check to see if this probability is greater than 50%.  If so continue selecting 'Random'.  Otherwise, begin selecting 'Next' until you arrive at the desired track

In order the impliment this stradegy, we need a way of calculating the probability that continuing to use 'Random' will get us to the desired track faster than beginning to use 'Next'

Based on the description above, we are looking for 

$$ \text{Total Probability} = \sum_{i=1}^{d}P(D^{(i)} \le d-i)$$

In the expression above, $D^{(i)}$ is the minimum distance to the desired track of i uniformly distributed samples and $d$ is the distance from the current track to the desired track. 

Each term in the summation above can be evaluated as follows

$$P(D^{(i)} \le d-i) = P(min(D_1, \cdots , D_i) \le d-i)$$

Rewriting in terms of the inverse yields

$$P(min(D_1, \cdots , D_i) \le d-i) = 1-P(min(D_1, \cdots , D_i) \gt d-i)$$

If the minimum of all $D_1, \cdots, D_i$ is less than $d-i$, then all $D_1, \cdots, D_i$ is less than $d-i$.  And since each $D$ is independent, we can rewrite the expression above as follows

$$1-P(min(D_1, \cdots , D_i) \gt d-i) = 1-P(D_1 \gt d-i) * \cdots * P(D_i \gt d-i)$$

Rewriting this expression with each probability term written in terms of its inverse, we get the following expression

$$1-P(D_1 \gt d-i) * \cdots * P(D_i \gt d-i) = 1-[(1-P(D_1 \le d-i)) * \cdots * (1-P(D_i \le d-i))$$

Since each $D$ is identically distributed we can rewrite this as

$$1-[1-P(D \le d-i)]^i$$

The probability that any given uniform random sample from the tracks is less than $d-i$ is $\frac{d-i}{100}$ so the expression above can be rewritten as follows

$$1-\left[1-\frac{d-i}{100}\right]^i$$

Plugging this expression into the summation we get

$$\text{Total Probability} = \sum_{i=1}^{d}P(D^{(i)} \le d-i) = \sum_{i=1}^{d} 1-\left[1-\frac{d-i}{100}\right]^i$$

Equipped with this expression, all we need to do is evaluate this probability at each random track we land on until we get a probability less than 50%.  At that time, we will begin using the 'Next' botton to finish getting to the desired track.  

### Expected number steps to get to desired track
I'm not quite good enought at probability to calculate this analytically so I will cheat and use a Monte Carlo simulation to approximate the average number of steps needed to get to the desired track using the stradegy described above. 

In [ ]:
import random

def distance_to_track(curr_track, desired_track):
    if curr_track <= desired_track:
        return desired_track - curr_track
    else:
        return desired_track +(100-curr_track)

def prob_random_better(dist_to_track):
    return sum([1-(1-(d-i)/100)**i for i in range(1,dist_to_track+1)])

def run_simulation(desired_track):
    